In [1]:
!pip install transformers sentencepiece


In [2]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Choose model
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)


Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
# TESTING

import time

job_ad = "We are looking for a software developer to join our team in our downtown office in New York. The salary will be a daily rate of 130 USD"

# Format the prompt

prompt = f"Classify the work arrangement of the following job ad: {job_ad}"

start_time = time.time()
# Tokenize
input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True).to(device) # Move input_ids to the same device as the model

# Generate output
job_ids = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)

stop_time = time.time()
execution_time = stop_time - start_time
print("Execution time:", execution_time, "seconds")

# Decode and print
print("Summary:", tokenizer.decode(job_ids[0], skip_special_tokens=True))

Execution time: 1.3475978374481201 seconds
Summary: in our downtown office in New York.


In [3]:
import os
from google.colab import drive

drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/AB_job_data_files'

import os
from google.colab import drive

# find location of different files:
file_location = {}
# Check if the folder exists
if os.path.exists(folder_path):
  os.chdir(folder_path)
  for num, f in enumerate(os.listdir()):
    file_location[f] = os.path.join(folder_path, f)

else:
  print(f"Folder not found: {folder_path}")

print(file_location)

Mounted at /content/drive
{'seniority_labelled_test_set.csv': '/content/drive/MyDrive/AB_job_data_files/seniority_labelled_test_set.csv', 'unlabelled_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/unlabelled_development_set.csv', 'salary_labelled_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/salary_labelled_development_set.csv', 'seniority_labelled_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/seniority_labelled_development_set.csv', 'work_arrangements_test_set.csv': '/content/drive/MyDrive/AB_job_data_files/work_arrangements_test_set.csv', 'salary_labelled_test_set.csv': '/content/drive/MyDrive/AB_job_data_files/salary_labelled_test_set.csv', 'work_arrangements_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/work_arrangements_development_set.csv', 'results': '/content/drive/MyDrive/AB_job_data_files/results', 'wandb': '/content/drive/MyDrive/AB_job_data_files/wandb', 't5-small-work-arrangement-finetuned': '/content/d

In [4]:
def salary_estimate(job_ad):

    # Format the prompt
    prompt = f"Identify the salary from the job ad in the format minimum-maximum-currency-frequency: {job_ad}"

    # Tokenize
    input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True).to(device) # Move input_ids to the same device as the model

    # Generate output
    job_ids = model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)

    # Decode and return
    return tokenizer.decode(job_ids[0], skip_special_tokens=True)

In [13]:
# DEV set
import pandas as pd

salary_dev = pd.read_csv(file_location['salary_labelled_development_set.csv'])
salary_dev.head()

results = 0
count = 0
for index, row in salary_dev.iterrows():
    predicted_label = salary_estimate(row['job_ad_details'])

    true_label =  row['y_true']

    print(predicted_label, '----', true_label)
    print(predicted_label == true_label)
    if predicted_label == true_label:
        results += 1
    count += 1

print("count:", count)
print("result:", results)
print("correctly indentified:", results/count)


/p> /div>/div>/div> div>div>/div> div>div>div> Date Posted p> 2022-10-27 /p> /div>/div> div>div>/div> div>div>div>div> Date Posted p> 2022 ---- 17500-17500-PHP-MONTHLY
False
p>2023-08-09/p> strong>Aspiring Call Center Agents - Work from Home - Must be residing in Davao/strong>br /> /div> div> /div> div> /div> div> p>b>Job Location/b>/p> ---- 16000-16000-PHP-MONTHLY
False
/strong>/p>p>strong>You must be double vaccinated and have the right to live and work in New Zealand to apply for this role./strong>/strong>/p>p>strong>You must be double vaccinated and have the right to live and work in New Zealand to apply for this role./strong>/strong>/strong>/p> ---- 0-0-None-None
False
/li>li>Researching Payer websites for content related to Healthcare Revenue Cycle Management and Billing./li>li>Tracking maintenance – resolve any tracking errors that may arise. Monitor all tracking to ensure the latest versions of URLs/PDFs are input on a routine basis./li>li>Identify the salary from the job ad in

In [5]:
# TEST set
import pandas as pd

salary_test = pd.read_csv(file_location['salary_labelled_test_set.csv'])

results = 0
count = 0
for index, row in salary_test.iterrows():
    predicted_label = salary_estimate(row['job_ad_details'])

    true_label =  row['y_true']

    #print(work_arrang(row['job_ad']), '----', row['y_true'])
    #print(predicted_label == true_label)
    if predicted_label == true_label:
        results += 1
    count += 1

print("count:", count)
print("result:", results)
print("correctly indentified:", results/count)


count: 567
result: 0
correctly indentified: 0.0
